# Método de momentos

In [3]:
import numpy as np

## Cilindro conductor

Se sigue el método descripto en el capítulo 2 del libro _The method of moments in electromagnetics_, W. C. Gibson.

<div><image src="img/c2_cilindro.png"  width="400"></div>

<div><image src="img/c2_cilindro_discretizado.png"  width="500"></div>


In [12]:
def Zmn(row, m, a, delta):
    xm = delta/2 + (m-1) * delta
    num = row * delta - xm

    return num

In [14]:
w = np.array(np.arange(1,10))
Zmn(w, 1, 2, 2)

array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17.])

In [3]:
a = np.array([[1, 2], [3, 5]])

b = np.array([1, 2])

x = np.linalg.solve(a, b)

x

array([-1.,  1.])

In [5]:
# Check that the solution is correct:
np.allclose(np.dot(a, x), b)

True

In [ ]:
a = 0.125
h = 1
V = 0.5
item = np.array([], dtype=np.int32).reshape(0,4)
for i in range(4):
    for j in range(4):
        xi = -0.5 + a*(1+2*i)
        yj = -0.5 + a*(1+2*j)
        temp = np.array([xi,yj, h, V]).reshape(1,4)
        item = np.concatenate((item,temp))

h = 2
V = -0.5
for i in range(4):
    for j in range(4):
        xi = -0.5 + a*(1+2*i)
        yj = -0.5 + a*(1+2*j)
        temp = np.array([xi,yj, h, V]).reshape(1,4)
        item = np.concatenate((item,temp))
item

$$
L_{i,j}\approx \frac{k}{r + [L_a \cdot (1 + r/a)]^{-1}}
$$
where $a=b$ (squares), and
$$
r = \sqrt{(x_i-x_j)^2 + (y_i-y_j)^2 + (z_i-z_j)^2}
$$